In [1]:
ls ../data/navy/SentencesNAVY/

APR2014.csv      Dec2013.pdf      Jan2014.pdf      NOV2015.pdf
APR2015.pdf      FEB16.pdf        July2014.pdf     NOV2016.pdf
APR2016.pdf      FEB2015.pdf      July2015.pdf     NOV2017.pdf
APR2017.pdf      FEB2017.pdf      Jun2015.pdf      Nov2013.pdf
AUG2014.pdf      Feb2014.pdf      June2014.pdf     OCT2015.pdf
AUG2015.pdf      JAN2013.pdf      MAR2015.pdf      OCT2016.pdf
AUG2016.pdf      JAN2015.pdf      MAR2016.pdf      OCT2017.pdf
AUG2017.pdf      JAN2016.pdf      MAR2017.pdf      Oct2013.pdf
Apr2014.pdf      JAN2017.pdf      MAY2016.pdf      October2014.pdf
Apr2015.csv      JUL2016.pdf      MAY2017.pdf      SEPT2014.pdf
Aug2013.pdf      JUL2017.pdf      Mar2014.pdf      SEPT2015.pdf
DEC14.pdf        JULY2013.pdf     May2014.pdf      SEPT2016.pdf
DEC2016.pdf      JUN2016.pdf      May2015.pdf      SEPT2017.pdf
Dec15.pdf        JUN2017.pdf      NOV14.pdf        Sept2013.pdf


In [2]:
import textract
import sys
sys.path.append('utils/')
import navy_python_scraper
import glob
import random
import re
import pandas as pd

In [4]:
reload(navy_python_scraper)

<module 'navy_python_scraper' from 'utils/navy_python_scraper.py'>

In [5]:
data_dir = '../data/navy/SentencesNAVY/'
df_list = []
court_martial_list = []
for document in glob.glob('../data/navy/SentencesNAVY/*.pdf'):

    # naming...
#     print('----------------------------------')
#     print('processing %s...' % document)
#     print('----------------------------------')
    fp = document
    text = textract.process(fp)
    court_martial=unicode(text, encoding='utf-8').split('At a')[1:]
    court_martial_list += court_martial

    for idx, paragraph in enumerate(court_martial):
        if 'sentencing rehearing' not in paragraph:
            output = {}
            output['Document'] = document
            output['paragraph'] = paragraph
            paragraph= navy_python_scraper.clean_paragraph(paragraph)
            output = navy_python_scraper.get_date(paragraph, output)    
            
            output = navy_python_scraper.get_gender(paragraph, output)
            output = navy_python_scraper.get_rank(paragraph, output)
            output = navy_python_scraper.get_location(paragraph, output)
            output = navy_python_scraper.get_name(paragraph, output)
            output = navy_python_scraper.get_described_offense(paragraph, output)
            output = navy_python_scraper.get_conviction(paragraph, output)
            output = navy_python_scraper.get_sentence(paragraph, output)
            output = navy_python_scraper.cleanup(paragraph, output)
            df_list.append(output)

#     print
#     print

 General Court-Martial in Norfolk, Virginia, an E-5 and an E-4 were tried, jointly, for sexual assault.  On 14 April 2016, the panel of members returned a verdict of not guilty for both Sailors.        
 re-sentencing hearing for a previously held General Court-Martial in San Diego, CA, CTT1 Durville Christopher, USN was sentenced for indecent acts with a child and assault consummated by a battery on a child under the age of 16 years old.  On 25 March 2016, the panel of members sentenced him to be discharged with a Dishonorable Discharge, reduction in rank to paygrade E-1, and confinement for 2 years.                Special Court-Martial    None.   Navy Region Mid-Atlantic            General Court-Martial     
 re-sentencing hearing for a previously held General Court-Martial in Mayport, Florida, ABE2 Alonzo Bass, III, USN was sentenced for violations of a general order and wrongful sexual contact.  On 28 March 2016, the military judge sentenced him to be discharged with a Bad Condu

In [7]:
df = pd.DataFrame(df_list)

In [9]:
df.to_csv('../data/navy/navy-processed-alex.csv', encoding='utf8')